<a href="https://colab.research.google.com/github/Akil-Rajendran/VPT-Contest/blob/main/evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IMPORTING REQUIRED LIBRARIES**

In [6]:
pip install Open-Tamil

In [7]:
import os
import re 
from tamil import utf8
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
from nltk.tokenize import sent_tokenize

# **PREPROCESSING THE RAW DATA**

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **FUNCTON FOR CLEANING**


In [74]:
def clean(data):
  string = re.split("[\n\.]",data)
  arr = []

  for i in string:
    if re.search("Parva", i):
      continue
    arr.append(i)

  string = ".".join(arr)
  string = re.sub("(\(?SECTION\s*[IVXLC]+\)?)|(p\.\s+\d+)|(if)|(Footnotes)|(\")|(\')", "", string)
  string = re.sub("\{.*?\}", "", string)
  string = re.sub("\(.*?\)", "", string)
  string.replace('i\.e\.,-\n',"")

  return string

### **RETRIEVING TEXTS FROM DRIVE**

In [75]:
def openfile(i, lang):
  temp = []
  file = []
  if lang == 'en':
    with open("/content/drive/MyDrive/resources/Mahabharatha-Adiparva-Section" + str(i) + "-en.txt", "r", encoding='utf8') as tam:
      temp = tam.read()
    file.append(clean(temp))
    
  elif lang == 'ta':
    with open("/content/drive/MyDrive/resources/Mahabharatha-Adiparva-Section" + str(i) + "-ta.txt", "r") as en:
      temp = en.read()
    file.append(clean(temp))
  return file

In [136]:
english = openfile(27, 'en')
tamil = openfile(27, 'ta')

In [77]:
tamil

['சௌதி சொன்னார், மழையில் நனைந்த பாம்புகள், அதன்பிறகு மிகவும் மகிழ்ந்திருந்தன. அந்த அழகான இறகுகள் கொண்ட பறவையானவன்  தங்களைச் சுமந்து செல்ல விரைவாகத் தீவை அடைந்தனர். அந்தத் தீவு, மகரங்களின் இருப்பிடமாக, அண்டம் படைத்தோனால் தீர்மானிக்கப்பட்டிருந்தது. அங்கே அவர்கள் பயங்கரமான லவணச் சமுத்திரத்தைக்  கண்டனர். கருடனுடன் வந்தவர்கள் அங்கே நீரால் சுத்தம் செய்யப்பட்ட ஓர் அழகான காடு கடலின் மடியில் இருந்ததையும், இறகுகள் கொண்ட இசைக்கலைஞர்களின்  இன்னிசை எங்கும் நிறைந்து இருந்ததையும் உணர்ந்தனர்.  அக்கானகத்தில் கருடனுடன் அப்பாம்புகள் வந்திறங்கினர். அங்கே, கொத்துக் கொத்தாக மரங்கள் பலவகைப்பட்ட மலர்களுடனும் பழங்களுடனும் நிறைந்து இருந்தன..அழகான மாளிகைகளும் அங்கு நிறைந்திருந்தன. தாமரை நிறைந்த தடாகங்களாலும் நிறைந்திருந்தன. சுத்தமான நீர் கொண்ட பல ஏரிகள் அங்கு இருந்தன. அங்குத் தவழ்ந்த சுத்தமணங்கொண்ட தென்றலால் உற்சாகம் நிலவியது. மலய மலையில் மட்டுமே விளையும் பல மரங்கள் அங்கே இருந்தன. அவற்றின் உயரங்கள் விண்ணை முட்டும் அளவுக்கு இருந்தன. இன்னும் பல மரங்களில், அவற்றின் மலர்கள் உதிர்ந்து தென்றலின் உதவியால் அழகாகப் பரவிக

In [68]:
english

['Sauti said, And then the Nagas drenched by that shower, became exceedingly glad And borne by that bird of fair feathers, they soon arrived at the island That island had been fixed by the Creator of the Universe as the abode of the makaras There they saw the terrible Lavana Samudra  On arriving there with Garuda, they saw there a beautul forest washed by the waters of the sea and resounding with the music of winged choirs And there were clusters of trees all around laden with various fruits and flowers And there were also fair mansions all around; and many tanks full of lotuses And it was also adorned with many lakes of pure water And it was refreshed with pure incense-breathing breezes And it was adorned with many a tree that grew only on the hills of Malaya, and seemed by their tallness to reach the very heavens And there were also various other trees whose flowers were scattered all around by the breeze And that forest was charming and dear to the Gandharvas and always gave them pl

# **MODEL INSTALLATION AND TRAINING**

In [ ]:
# clone the repo for running evaluation
!git clone https://github.com/AI4Bharat/indicTrans.git
%cd indicTrans
# clone requirements repositories
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/rsennrich/subword-nmt.git
%cd ..

In [ ]:
# Install the necessary libraries
!pip install sacremoses pandas mock sacrebleu tensorboardX pyarrow indic-nlp-library
! pip install mosestokenizer subword-nmt
# Install fairseq from source
!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
# !git checkout da9eaba12d82b9bfc1442f0e2c6fc1b895f4d35d
!pip install --editable ./

%cd ..

In [1]:
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils

In [ ]:
# downloading the indic-en model
!wget https://storage.googleapis.com/samanantar-public/V0.2/models/indic-en.zip
!unzip indic-en.zip

%cd indicTrans

In [ ]:
from indicTrans.inference.engine import Model

indic2en_model = Model(expdir='../indic-en')

# **MACHINE TRANSLATION**

In [84]:
eng_trans = []
eng_trans = indic2en_model.translate_paragraph(tamil[0], 'ta', 'en')

100%|██████████| 24/24 [00:00<00:00, 2797.21it/s]


In [129]:
english[0]

'.Sauti said, And then the Nagas drenched by that shower, became exceedingly glad. And borne by that bird of fair feathers, they soon arrived at the island. That island had been fixed by the Creator of the Universe as the abode of the makaras. There they saw the terrible Lavana Samudra . On arriving there with Garuda, they saw there a beautul forest washed by the waters of the sea and resounding with the music of winged choirs. And there were clusters of trees all around laden with various fruits and flowers. And there were also fair mansions all around; and many tanks full of lotuses. And it was also adorned with many lakes of pure water. And it was refreshed with pure incense-breathing breezes. And it was adorned with many a tree that grew only on the hills of Malaya, and seemed by their tallness to reach the very heavens. And there were also various other trees whose flowers were scattered all around by the breeze. And that forest was charming and dear to the Gandharvas and always g

In [130]:
eng_trans

'Saudis said, the snakes which were wet in the rain, were very happy after that. The beautiful winged bird rushed to the island to carry itself. The island was determined by the Creator of the universe as the location of the pollinators. There they saw a terrible sea of salt. Those who came with Garuda felt that there was a beautiful underwater forest on the seabed and that the music of the feathered musicians was everywhere. In the same way, we have been able to bring about a change in people\'s lives. There, in clusters, trees were abundant with a variety of flowers and fruits, and there were beautiful mansions. It was also filled with lotus lakes. There were many lakes with clean water. There was excitement in the air because of the fresh air. There were many trees that grew only in the Malayan hills. They were tall enough to reach the sky. In many other trees, their flowers fell and were spread beautifully with the help of a thistle. In this way, the forest was beautiful, always pl

# **BLEU SCORE CALCULATION**

In [132]:
import sacrebleu
from sacremoses import MosesDetokenizer
md = MosesDetokenizer(lang='en')

# Open the test dataset human translation file and detokenize the references
refs = [] 
refs.append(english[0])
print("Reference 1st sentence:", refs[0])

# Open the translation file by the NMT model and detokenize the predictions
preds = [] 
preds.append(eng_trans)
print("Model Translation 1st sentence :", preds[0])

Reference 1st sentence: .Sauti said, And then the Nagas drenched by that shower, became exceedingly glad. And borne by that bird of fair feathers, they soon arrived at the island. That island had been fixed by the Creator of the Universe as the abode of the makaras. There they saw the terrible Lavana Samudra . On arriving there with Garuda, they saw there a beautul forest washed by the waters of the sea and resounding with the music of winged choirs. And there were clusters of trees all around laden with various fruits and flowers. And there were also fair mansions all around; and many tanks full of lotuses. And it was also adorned with many lakes of pure water. And it was refreshed with pure incense-breathing breezes. And it was adorned with many a tree that grew only on the hills of Malaya, and seemed by their tallness to reach the very heavens. And there were also various other trees whose flowers were scattered all around by the breeze. And that forest was charming and dear to the 

In [133]:
for line in zip(refs,preds):
    test = line[0]
    pred = line[1]

In [134]:
print(test, "\t--->\t", pred)

.Sauti said, And then the Nagas drenched by that shower, became exceedingly glad. And borne by that bird of fair feathers, they soon arrived at the island. That island had been fixed by the Creator of the Universe as the abode of the makaras. There they saw the terrible Lavana Samudra . On arriving there with Garuda, they saw there a beautul forest washed by the waters of the sea and resounding with the music of winged choirs. And there were clusters of trees all around laden with various fruits and flowers. And there were also fair mansions all around; and many tanks full of lotuses. And it was also adorned with many lakes of pure water. And it was refreshed with pure incense-breathing breezes. And it was adorned with many a tree that grew only on the hills of Malaya, and seemed by their tallness to reach the very heavens. And there were also various other trees whose flowers were scattered all around by the breeze. And that forest was charming and dear to the Gandharvas and always ga

In [135]:
bleu = sacrebleu.sentence_bleu(pred, [test], smooth_method='exp')
print(bleu.score)

6.432976148394318
